In [14]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pandas as pd
import pandas as pd
from pandas_datareader import data
import yfinance as yf

In [15]:
# input
symbols = ['AAPL','MSFT','AMD','NVDA','GE']
start = '2012-01-01'
end = '2019-09-11'

# Read data 
dataset = yf.download(symbols,start,end)['Close']

# View Columns
close = dataset.fillna(0)

[*********************100%***********************]  5 of 5 completed

1 Failed download:
- AMD: No data found for this date range, symbol may be delisted


In [16]:
cl = close.round(2)

In [17]:
cl

,AAPL,AMD,GE,MSFT,NVDA
Date,,,,,
2012-01-03,14.69,0.0,141.23,26.77,3.51
2012-01-04,14.77,0.0,142.77,27.40,3.55
2012-01-05,14.93,0.0,142.69,27.68,3.68
2012-01-06,15.09,0.0,143.46,28.11,3.63
2012-01-09,15.06,0.0,145.08,27.74,3.63
...,...,...,...,...,...
2019-09-04,52.30,0.0,70.40,137.63,42.19
2019-09-05,53.32,0.0,70.48,140.05,44.94
2019-09-06,53.31,0.0,69.68,139.10,44.66


In [18]:
n = 'MSFT'

In [51]:
cl.loc[cl['AMD'] == 5.48]['GE'] = 2

/Users/kexin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [34]:
close['AMD'] == 5.48

Date
2012-01-03    False
2012-01-04    False
2012-01-05    False
2012-01-06    False
2012-01-09    False
              ...  
2019-09-04    False
2019-09-05    False
2019-09-06    False
2019-09-09    False
2019-09-10    False
Name: AMD, Length: 1934, dtype: bool

In [27]:
close

,AAPL,AMD,GE,MSFT,NVDA
Date,,,,,
2012-01-03,14.686786,5.480000,141.230774,26.770000,3.510000
2012-01-04,14.765714,5.460000,142.769226,27.400000,3.550000
2012-01-05,14.929643,5.460000,142.692307,27.680000,3.677500
2012-01-06,15.085714,5.430000,143.461533,28.110001,3.635000
2012-01-09,15.061786,5.590000,145.076920,27.740000,3.635000
...,...,...,...,...,...
2019-09-04,52.297501,30.950001,70.400002,137.630005,42.189999
2019-09-05,53.320000,31.500000,70.480003,140.050003,44.935001
2019-09-06,53.314999,30.559999,69.680000,139.100006,44.662498


['G', 'E']

In [10]:
a = [1,2,3,4]
b = [1,2]
list(set(a)-set(b))

[3, 4]

In [13]:
a+b

[1, 2, 3, 4, 1, 2]

In [61]:
def best_combination(pnl_collated, stats, criteria):
    min_sharpe = criteria['Sharpe']
    max_turnover = criteria['Turnover']
    stats_sorted = stats.sort_values(by=['OS:Sharpe'], ascending = False)
    num_alphas = stats.shape[0]
    sorted_alphas = stats[['id']]
    curr_best_combi = [stats_sorted.iloc[[0]]['id'][0]]
#     stats_sorted.iloc[[0]]['PNL'][0]
#     curr_alphas = [stats_sorted.iloc[[0]]['id'][0]]
    count = 1
    #curr_sharpe = stats.loc[stats['alpha_name'] == sorted_alphas[0]]['OS:Sharpe'].item()
    curr_sharpe = stats_sorted.iloc[[0]]['OS:Sharpe'][0]
    curr_turnover = stats_sorted.iloc[[0]]['OS:turnover'][0]
    while count < num_alphas:
        find_best = list(set(sorted_alphas)-set(curr_best_combi))
        temp = curr_best_combi.copy()
        for alpha in find_best:
            curr_pnl = pnl_collated[curr_best_combi + [alpha]].sum(axis=1)/(len(curr_best_combi)+1)
            if get_sharpe(curr_pnl) > curr_sharpe and get_sharpe(curr_pnl) > min_sharpe:
                #find correlation and check top is less than threshold
                curr_sharpe = get_sharpe(curr_pnl)
                curr_turnover = get_turnover(curr_pnl)
                if len(temp) > 1:
                    temp.pop()
                    temp.append(alpha)
                else:
                    temp.append(alpha)
            elif get_sharpe(curr_pnl) == curr_sharpe and get_turnover(curr_pnl) < curr_turnover and get_turnover(curr_pnl) < max_turnover:
                curr_sharpe = get_sharpe(curr_pnl)
                curr_turnover = get_turnover(curr_pnl)
                if len(temp) > 1:
                    temp.pop()
                    temp.append(alpha)
                else:
                    temp.append(alpha)
        
        curr_best_combi = temp
        count += 1
    best_sharpe = get_overall(curr_pnl)
    final_indiv_pnl = pnl_collated[curr_best_combi]
    final_pnl = final_indiv_pnl.sum(axis=1)
    return curr_best_combi, final_indiv_pnl, best_sharpe

In [60]:
def best_combo_of(pnl_collated, stats, threshold, num=2):
    tats_sorted = stats.sort_values(by=['OS:Sharpe'], ascending = False)
    sorted_alphas = stats[['alpha_name']]
    curr_best_combi = [sorted_alphas[0]]
    count = 1
    curr_sharpe = stats.loc[stats['alpha_name'] == sorted_alphas[0]]['OS:Sharpe'].item()
    while count <= num:
        find_best = list(set(sorted_alphas)-set(curr_best_combi))
        temp = curr_best_combi.copy()
        for alpha in find_best:
            curr_pnl = pnl_collated[curr_best_combi + [alpha]].sum(axis=1)
            if get_sharpe(curr_pnl) > curr_sharpe and get_sharpe(curr_pnl) > threshold:
                curr_sharpe = get_sharpe(curr_pnl)
                if len(temp) > 1:
                    temp.pop()
                    temp.append(alpha)
                else:
                    temp.append(alpha)
            elif get_sharpe(curr_pnl) == curr_sharpe and get_turnover(positions) > curr_turnover:
                curr_turnover = get_turnover(positions)
                if len(temp) > 1:
                    temp.pop()
                    temp.append(alpha)
                else:
                    temp.append(alpha)
            elif get_sharpe(curr_pnl) == curr_sharpe and get_turnover(positions) == curr_turnover and get_mdd(curr_position) < curr_mdd:
                curr_mdd = get_mdd(curr_position)
                if len(temp) > 1:
                    temp.pop()
                    temp.append(alpha)
                else:
                    temp.append(alpha)
        curr_best_combi = temp
        count += 1
    best_sharpe = get_overall(curr_pnl)
    return curr_best_combi, curr_pnl, best_sharpe

In [22]:
cl.iloc[[0]]['AAPL'][0]

14.69

In [9]:
x = [1,2,3]
y = [2,3,4]

In [12]:
corr = scipy.stats.pearsonr(x, y)[0]

In [6]:
import scipy.stats

In [11]:
corr

(0.9999999999999998, 1.3415758552508151e-08)

In [23]:
cl.sum(axis=1)

Date
2012-01-03    186.20
2012-01-04    188.49
2012-01-05    188.98
2012-01-06    190.29
2012-01-09    191.51
               ...  
2019-09-04    302.52
2019-09-05    308.79
2019-09-06    306.75
2019-09-09    307.86
2019-09-10    309.16
Length: 1934, dtype: float64

In [24]:
cl

,AAPL,AMD,GE,MSFT,NVDA
Date,,,,,
2012-01-03,14.69,0.0,141.23,26.77,3.51
2012-01-04,14.77,0.0,142.77,27.40,3.55
2012-01-05,14.93,0.0,142.69,27.68,3.68
2012-01-06,15.09,0.0,143.46,28.11,3.63
2012-01-09,15.06,0.0,145.08,27.74,3.63
...,...,...,...,...,...
2019-09-04,52.30,0.0,70.40,137.63,42.19
2019-09-05,53.32,0.0,70.48,140.05,44.94
2019-09-06,53.31,0.0,69.68,139.10,44.66
